### HRA API Use Case Documentation

[Click here](hra-api-client-usage.ipynb) to view the installation and general documentation for hra-api python module.

In this notebook we will demonstrate one particular use case for which the HRA API module can be utilized. 

## Comments

 - Andi added emojis to headings and INDEX below where &#x2705; = success, &#x274C; = issue, &#x2753; = uncertain if outcome is expected, &#128679; = in progress.
 - An error log is [here](README.md).

## INDEX

1. [Import HRA API module](#import_client) &#x2705;
2. [Imports](#import_packages) &#x2705;
3. [Configuration](#configuration) &#x2705;
4. [Check database status](#status) &#x2705;
5. [Using kidney](#kidney) &#128679;


<a id='import_client'></a>
### We first import all the necessary packages from the hra_api_client. &#x2705;

In [1]:
import hra_api_client
from hra_api_client.api import v1_api as default_api

<a id='import_packages'></a>
### Import all other packages required to produce output as expected. &#x2705;
The pandas module is used for a dataframe, which is used for displaying and aggregating. <br>
The csv module is used for writing all the data into a csv file.

In [2]:
import pandas as pd
import time
from pprint import pprint

<a id='configuration'></a>
### Configuration &#x2705;
You'll need to point the host in the configuration to the instance you'd like to work with. 
More Info [here](https://github.com/hubmapconsortium/ccf-ui/blob/main/ccf-api-usage.ipynb).

In [3]:

configuration = hra_api_client.Configuration(
    host = "https://apps.humanatlas.io/api"
)
api_client = hra_api_client.ApiClient(configuration)

api_instance = default_api.V1Api(api_client)

<a id='status'></a>
### Check Database Status &#x2705;
This is a optional step which can be used to reduce wait times for other methods.

In [4]:
# replace dict notation with dot notation
db_ready = False
result = None
while not db_ready:
    result = api_instance.db_status()
    pprint(result)
    if result.status == 'Ready':
        db_ready = True
    else:
        print('Database not ready yet! Retrying...', result)
        time.sleep(2)
print('Database ready!\n', result)

DatabaseStatus(status='Ready', checkback=3600000, load_time=22594, message='Database successfully loaded')
Database ready!
 status='Ready' checkback=3600000 load_time=22594 message='Database successfully loaded'


<a id='kidney'></a>
### Our use case is using Kidney as the reference organ. &#128679; 
We are using the ontology link for kidney directly: http://purl.obolibrary.org/obo/UBERON_0002113

The `scene` method gives us a list of all the organs and tissue blocks that make up a 3D scene. &#x2705;

In [5]:
sex = "both"
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0002113"]
sceneResult = None
try:
    sceneResult = api_instance.scene(sex=sex, ontology_terms=ontology_terms)
    pprint(sceneResult)
except hra_api_client.ApiException as e:
    print("Exception when calling DefaultApi->scene: %s\n" % e)

[SpatialSceneNode(name=None, tooltip=None, unpickable=None, geometry=None, lighting=None, zoom_based_opacity=None, zoom_to_on_load=None, scenegraph=None, scenegraph_node=None, color=None, opacity=None, transform_matrix=None, priority=None, entity_id=None, ccf_annotations=None, representation_of='http://purl.obolibrary.org/obo/UBERON_0002097', reference_organ='https://purl.humanatlas.io/ref-organ/skin-female/v1.4#primary', sex='Female', additional_properties={'@id': 'https://purl.humanatlas.io/ref-organ/skin-female/v1.4#primary', '@type': 'SpatialSceneNode', 'tooltip': 'skin', 'scenegraph': 'https://cdn.humanatlas.io/digital-objects/ref-organ/skin-female/v1.4/assets/3d-vh-f-skin.glb', 'scenegraphNode': 'VH_F_skin', 'transformMatrix': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0.4, 0, 0, 1], 'color': [255, 255, 255, 255], 'opacity': 0.5, 'unpickable': True, '_lighting': 'pbr', 'zoomBasedOpacity': False}),
 SpatialSceneNode(name=None, tooltip=None, unpickable=None, geometry=None, lighting=None,

We convert the `sceneResult` into a dictionary where the keys are the `entity_id`s of the tissue blocks and the values are the anatomical structures in the kidney the tissue blocks collide with. &#x2705;

In [6]:
sceneEntityASDict = {}
for scene in sceneResult:
    if scene.entity_id == None:
        continue
    if scene.entity_id not in sceneEntityASDict:
        sceneEntityASDict[scene.entity_id] = []
    sceneEntityASDict[scene.entity_id].extend(scene.ccf_annotations)

pprint(sceneEntityASDict)

{'http://dx.doi.org/10.1681/ASN.2016091027#Donor1_TissueBlock1': ['http://purl.obolibrary.org/obo/UBERON_0013702',
                                                                  'http://purl.obolibrary.org/obo/UBERON_0002113',
                                                                  'http://purl.obolibrary.org/obo/UBERON_0004539'],
 'http://dx.doi.org/10.1681/ASN.2016091027#Donor2_TissueBlock1': ['http://purl.obolibrary.org/obo/UBERON_0013702',
                                                                  'http://purl.obolibrary.org/obo/UBERON_0002113',
                                                                  'http://purl.obolibrary.org/obo/UBERON_0004539'],
 'http://dx.doi.org/10.1681/ASN.2016091027#Donor3_TissueBlock1': ['http://purl.obolibrary.org/obo/UBERON_0013702',
                                                                  'http://purl.obolibrary.org/obo/UBERON_0002113',
                                                                  'http://purl

We then use a SPARQL query to get the AS label, CT Label and the CT iri using the AS IRIs we obtained in the last step. &#x2705;

In [7]:
query = '''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ccf: <http://purl.org/ccf/>

SELECT DISTINCT (STR(?asLabel) as ?as_label) (STR(?qlabel) as ?cell_label) ?as_iri ?cell_iri WHERE {
  ?cell_iri ccf:ccf_located_in ?as_iri .
  ?cell_iri rdfs:label ?qlabel .
  ?as_iri rdfs:label ?asLabel .

  FILTER (?as_iri in (%s))

}'''

purlIris = set()

# Collecting all unique AS iri's

for key in sceneEntityASDict.keys():
    for value in sceneEntityASDict[key]:
        purlIris.add(value)

purlString = ", ".join("<" + purl + ">" for purl in purlIris)
queryResponse = None
query = query % purlString

try:
    queryResponse = api_instance.sparql(query=query, format='')
    pprint(queryResponse)

except hra_api_client.ApiException as e:

    print("Exception when calling DefaultApi->sparql: %s\n" % e)

{'head': {'vars': ['as_label', 'cell_label', 'as_iri', 'cell_iri']},
 'results': {'bindings': [{'as_iri': {'type': 'uri',
                                      'value': 'http://purl.obolibrary.org/obo/UBERON_0000362'},
                           'as_label': {'type': 'literal',
                                        'value': 'renal medulla'},
                           'cell_iri': {'type': 'uri',
                                        'value': 'http://purl.obolibrary.org/obo/CL_1000718'},
                           'cell_label': {'type': 'literal',
                                          'value': 'kidney inner medulla '
                                                   'collecting duct principal '
                                                   'cell'}},
                          {'as_iri': {'type': 'uri',
                                      'value': 'http://purl.obolibrary.org/obo/UBERON_0000362'},
                           'as_label': {'type': 'literal',
                   

We now reverse the sceneEntityASDict to now have AS: entity_id relationship &#x2705;

In [8]:
def reverseObject(input_object):
    reversed_object = {}

    for key, values in input_object.items():
        for value in values:
            if value not in reversed_object:
                reversed_object[value] = set()
            reversed_object[value].add(key)
    return reversed_object

ASEntityDict = reverseObject(sceneEntityASDict)
pprint(ASEntityDict)

{'http://purl.obolibrary.org/obo/UBERON_0000056': {'https://entity.api.hubmapconsortium.org/entities/11d794402e067e54888bb8a4af4184e0',
                                                   'https://entity.api.hubmapconsortium.org/entities/263854e0346766efa516bd69bfc1e035'},
 'http://purl.obolibrary.org/obo/UBERON_0000362': {'https://entity.api.hubmapconsortium.org/entities/015f9976f3fed4c35dbbaad622a413d9',
                                                   'https://entity.api.hubmapconsortium.org/entities/0a558711c6b9d5b9029717d722a2c31c',
                                                   'https://entity.api.hubmapconsortium.org/entities/0be6dee31365fd1e590a45263df043c6',
                                                   'https://entity.api.hubmapconsortium.org/entities/0ec930e0f4a8fcf12ea136a28e1acad4',
                                                   'https://entity.api.hubmapconsortium.org/entities/11d794402e067e54888bb8a4af4184e0',
                                               

Using the method `tissue_blocks`, we get donor label information for each `entity_id`. &#x2705;

In [9]:
sex = "both"
# We use the ontology term for "left kidney". Just using the term for "kidney" will result in an error.

ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0004538"] 

tissueBlockResult = None

try:
    tissueBlockResult = api_instance.tissue_blocks(sex=sex, ontology_terms=ontology_terms)
except hra_api_client.ApiException as e:

    print("Exception when calling DefaultApi->tissue_blocks: %s\n" % e)

# Convert the tissueBlockResult object to a list of dict
tissueBlockResultList = list(map(lambda b: b.to_dict(), tissueBlockResult))
pprint(tissueBlockResultList)

[{'@id': 'https://entity.api.hubmapconsortium.org/entities/039ce6e8d5fa342811a4b317846b7281',
  '@type': 'Sample',
  'datasets': [],
  'description': '35 x 19 x 3 millimeter, 3 millimeter, 1 Sections',
  'donor': {'@id': 'https://entity.api.hubmapconsortium.org/entities/2510644405f4e7fd86d31af0001b840f',
            '@type': 'Donor',
            'description': 'Entered 12/26/2019, Jamie Allen, TMC-Vanderbilt',
            'label': 'Male, Age 56, BMI 32.5',
            'link': 'https://portal.hubmapconsortium.org/browse/donor/2510644405f4e7fd86d31af0001b840f',
            'providerName': 'TMC-Vanderbilt'},
  'label': 'Registered 6/10/2020, Jamie Allen, TMC-Vanderbilt',
  'link': 'https://portal.hubmapconsortium.org/browse/sample/039ce6e8d5fa342811a4b317846b7281',
  'sampleType': 'Tissue Block',
  'sectionCount': 1,
  'sectionSize': 3,
  'sectionUnits': 'millimeter',
  'sections': [{'@id': 'https://entity.api.hubmapconsortium.org/entities/03dd86ef9ffd739b4cad8b99eb1e3fbf',
              

To make it easier to find the donor labels we convert the above result to a dictionary that related the entity_id to the donor_label. &#x2705;

In [10]:
blockDonorLabel = {}
for tissueBlock in tissueBlockResultList:
    blockDonorLabel[tissueBlock['@id']] = tissueBlock['donor']['label']

pprint(blockDonorLabel)

{'https://entity.api.hubmapconsortium.org/entities/039ce6e8d5fa342811a4b317846b7281': 'Male, '
                                                                                      'Age '
                                                                                      '56, '
                                                                                      'BMI '
                                                                                      '32.5',
 'https://entity.api.hubmapconsortium.org/entities/07c995983aac113b442c0dc0c0b42ae5': 'Male, '
                                                                                      'Age '
                                                                                      '53, '
                                                                                      'BMI '
                                                                                      '26.5',
 'https://entity.api.hubmapconsortium.org/entities/088adc477043f

<a id='export'></a>
### Export 	&#128679;

Now we create the report, using data from the above methods. &#x274C;

In [11]:
# Can't use set because - unhashable type: 'dict'
mergedData = []

for item in queryResponse:
    for result in queryResponse['results']['bindings']:
        # print(result)
        pass

pprint(ASEntityDict)

# for response in queryResponse:
#     pprint(queryResponse[response])
#     entity_ids = ASEntityDict.keys()
    # pprint(ASEntityDict.keys())
    # for e in entity_ids:
    #     newResponse = response.copy()
    #     newResponse['block_id'] = e
    #     newResponse['donor_label'] = blockDonorLabel[e]
    #     if newResponse not in mergedData:
    #         mergedData.append(newResponse)

{'http://purl.obolibrary.org/obo/UBERON_0000056': {'https://entity.api.hubmapconsortium.org/entities/11d794402e067e54888bb8a4af4184e0',
                                                   'https://entity.api.hubmapconsortium.org/entities/263854e0346766efa516bd69bfc1e035'},
 'http://purl.obolibrary.org/obo/UBERON_0000362': {'https://entity.api.hubmapconsortium.org/entities/015f9976f3fed4c35dbbaad622a413d9',
                                                   'https://entity.api.hubmapconsortium.org/entities/0a558711c6b9d5b9029717d722a2c31c',
                                                   'https://entity.api.hubmapconsortium.org/entities/0be6dee31365fd1e590a45263df043c6',
                                                   'https://entity.api.hubmapconsortium.org/entities/0ec930e0f4a8fcf12ea136a28e1acad4',
                                                   'https://entity.api.hubmapconsortium.org/entities/11d794402e067e54888bb8a4af4184e0',
                                               

This will save the output as a csv file. <br>
You can change the name of the file in first line of the next block. &#128679;

In [12]:
fileName = "exampleFileName"
import csv
def save_to_csv(data, headers, filename):
     with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the headers
        writer.writerow(headers)

        # Write the data
        for obj in data:
            row = [obj.get(headers[header], '') for header in headers]
            writer.writerow(row)

header_mapping = {
    'Block Id': 'block_id',
    'AS': 'as_iri',
    'AS Label': 'as_label',
    'CT': 'cell_iri',
    'CT Label': 'cell_label',
    'Donor Label':'donor_label'
}
save_to_csv(mergedData, header_mapping, f'${fileName}.csv')

In [13]:
df = pd.DataFrame(mergedData)
df.head(len(df))

""


In [14]:
# Summary -- texttt
AsCounts = df['as_iri'].value_counts()
CtCounts = df['cell_iri'].value_counts()
blockCounts = df['block_id'].value_counts()

print(f'SUMMARY:\n\
Number of Blocks: {len(blockCounts)},\n\
Number of Unique ASs identified: {len(AsCounts)},\n\
Number of Unique CTs identified: {len(CtCounts)}')

KeyError: 'as_iri'